#KLUE-TC(YNAT) dataset (2800개)

- Labeling Error - 1200개는 라벨을 임의로 바꿈
→ 올바른 라벨로 재맵핑 필요
- Random Noise - 1600개는 text에 노이즈를 추가. 임의의 char 중 20~80%를 랜덤으로 다른 아스키코드로 대체
→ 높은 노이즈를 가지는 text는 제외, 낮은 샘플은 복구가 필요
 사람이 개별 판단하지 말 것. 자동 detection이 가능하게 하여 수정하거나, 삭제하거나 진행


- 접근 방식
1. 노이즈 데이터를 분류, 정규식 기반 노이즈 정제
1. 노이즈 정제된 데이터를 기반으로 모델 학습, 라벨링 에러 데이터를 재라벨링 


## import dependencies 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

from . import filter

df = pd.read_csv('./data/train.csv')

## 시각화 유틸 함수들

데이터 프레임의 정보를 분석하는 함수들

In [ ]:
#데이터 프레임 정보 출력
def print_df(df):
    print(df.info())
    print(df.describe())
    print(df.head())
    
def target_plot(df):
    target_counts = df['target'].value_counts().sort_index()
    print(target_counts)

    # 시각화
    plt.figure(figsize=(10, 6))
    sns.barplot(x=target_counts.index, y=target_counts.values, palette='viridis')
    plt.title('Count of Each Target Value')
    plt.xlabel('Target Value')
    plt.ylabel('Count')
    plt.xticks(range(0, 7))  # target의 범위에 맞춰 x축 값 설정
    plt.show()    
    
def print_targets(df):
    # target 값에 따라 데이터프레임 정렬
    df = df.sort_values(by='target')

    # 각 target 값에 대해 샘플 출력
    target_values = df['target'].unique()

    for target in target_values:
        print(f"Target: {target}")
        samples = df[df['target'] == target].sample(n=10, random_state=42)
        display(samples)  # Jupyter Notebook에서 DataFrame을 보기 좋게 출력

## 데이터 전처리

### ascii 오염 데이터 추출/정규식 필터링

In [ ]:
# 노이즈 데이터를 추출
high_noise = filter.SpecialCharFilter.filter_noise(df)

# 노이즈 데이터의 텍스트 정규식 처리
high_noise['text'] = high_noise['text'].apply(lambda x: re.sub(r'[^가-힣A-Z\u4E00-\u9FFF\s0-9\.%㎜㎡]', '', x))

display(high_noise)

In [ ]:
# 결과를 CSV 파일로 저장
high_noise.to_csv('./data/noise_removed.csv', index=False, encoding='utf-8-sig')

### 라벨 균일화
다운샘플링을 통해 데이터 균일화 시도

In [ ]:
df = pd.read_csv('data/clean_ascii+df_eng_ger_combined.csv')
target_plot(df)

df = pd.read_csv('output (7).csv')
target_plot(df)

## 데이터 트리밍

In [ ]:
working_df = pd.read_csv('data/df_eng_ger_combined.csv')

# 각 라벨의 최소 개수 찾기
min_count = working_df['target'].value_counts().min()

# 각 클래스에서 최소 개수만큼 샘플링하여 새로운 데이터프레임 생성
df_balanced = working_df.groupby('target').apply(lambda x: x.sample(min_count)).reset_index(drop=True)

df_balanced.to_csv('output/traimmed.csv', index=False, encoding='utf-8-sig')